In [1]:
import os
import numpy as np
import pandas as pd
import folium
from folium import plugins
import geopandas
import argparse
import glob
import gpxpy
import geojson
import webbrowser
import numpy as np
import matplotlib.cm as cm
from scipy.signal import medfilt


In [2]:
# functions

def calc_dist_between_one_point_to_all_points(lon0, lat0, lon_all, lat_all): 
    lat0_rad    = np.radians(lat0)
    lon0_rad    = np.radians(lon0)
    lat_all_rad = np.radians(lat_all)
    lon_all_rad = np.radians(lon_all)

    delta_lat = lat_all_rad - lat0_rad
    delta_lon = lon_all_rad - lon0_rad

    # apply haversine formula
    a = (np.sin(delta_lat/2.0))**2.0 + np.cos(lat0_rad)*np.cos(lat0_rad)*((np.sin(delta_lon/2.0))**2.0)
    c = 2.0*np.arctan2(np.sqrt(a), np.sqrt(1.0-a))

    r = 6371.0*1000.0
    dist_all = r*c
    
    return(dist_all)


def calc_dist_between_two_coords(lon0, lat0, lon1, lat1): 
    lat0_rad = np.radians(lat0)
    lat1_rad = np.radians(lat1)
    lon0_rad = np.radians(lon0)
    lon1_rad = np.radians(lon1)

    delta_lat = lat1_rad - lat0_rad
    delta_lon = lon1_rad - lon0_rad

    # apply haversine formula
    a = (np.sin(delta_lat/2.0))**2.0 + np.cos(lat0_rad)*np.cos(lat1_rad)*((np.sin(delta_lon/2.0))**2.0)
    c = 2.0*np.arctan2(np.sqrt(a), np.sqrt(1.0-a))

    r = 6371.0*1000.0
    dist = r*c
    
    return(dist)

def calc_dist_from_coords(p1, p2): # distance between p1 and p2 [lat,lon] (in deg)
    lat1 = np.radians(p1[0])
    lat2 = np.radians(p2[0])
    lon1 = np.radians(p1[1])
    lon2 = np.radians(p2[1])

    delta_lat = lat2-lat1
    delta_lon = lon2-lon1

    # Haversine formula
    a = np.power(np.sin(delta_lat/2.0), 2)+np.cos(lat1)*np.cos(lat2)*np.power(np.sin(delta_lon/2.0), 2)
    c = 2.0*np.arctan2(np.sqrt(a), np.sqrt(1.0-a))

    dist = 6371e3*c

    return(dist)

def calc_dist_from_coordsPoint2Line(p0, p1, p2): # distance from p0 to line defined by p1 and p2 [lat,lon] (in deg)
    # Mercator projection
    P0 = np.array([np.radians(p0[1]), np.arcsinh(np.tan(np.radians(p0[0])))])*6371e3
    P1 = np.array([np.radians(p1[1]), np.arcsinh(np.tan(np.radians(p1[0])))])*6371e3
    P2 = np.array([np.radians(p2[1]), np.arcsinh(np.tan(np.radians(p2[0])))])*6371e3

    # distance from point to line
    dist = abs((P2[1]-P1[1])*P0[0]-(P2[0]-P1[0])*P0[1]+P2[0]*P1[1]-P2[1]*P1[0])/np.sqrt(np.power(P2[1]-P1[1], 2)+np.power(P2[0]-P1[0], 2)) # (from https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line#Line_defined_by_two_points)

    return(dist)

def RDP(data, epsilon): # Ramer–Douglas–Peucker algorithm
    if epsilon <= 0:
        return(data)

    dist_max = 0
    index = 0

    for i in np.arange(1, data.shape[0]):
        dist = calc_dist_from_coordsPoint2Line(data[i, :2], data[0, :2], data[-1, :2]) # needs a 2D projection, does not work with cross-track distance

        if dist > dist_max:
            index = i
            dist_max = dist

    if dist_max > epsilon:
        tmp1 = RDP(data[:index+1, :], epsilon)
        tmp2 = RDP(data[index:, :], epsilon)

        data_new = np.vstack((tmp1[:-1], tmp2))
    else:
        data_new = np.vstack((data[0, :], data[-1, :]))

    return(data_new)



In [3]:
dir_gpx_original  = 'data/gpx_original' 
dir_gpx_processed = 'data/gpx_processed' 
dir_geojson       = 'data/geojson'

dir_work = '/home/craigmatthewsmith/gps_tracks'
os.chdir(dir_work)
#os.getcwd()

#gpx_file_temp = os.path.join(dir_work, 'data_input_gpx/Afternoon_Run55.gpx')
#print(os.path.isfile(gpx_file_temp))

ingest_file_list = glob.glob(os.path.join(dir_gpx_processed, '*.gpx'))
n_files = len(ingest_file_list)
print('found %s files to process ' %(n_files))                              
#print('found %s files' %(n_files))                              
#geojson_file = os.path.join(data_geojson, '2020-03-22_15-06.geojson')
#os.path.isfile(geojson_file)



found 263 files to process 


In [4]:
use_RDP  = True
epsilon  = 1.0 # [m]
dist_min = 1.0 # 1.0, 75752 to 65536, 5386531 master.geojson file size 


In [5]:
lat_all = []
lon_all = []
ele_all = []

f = 10
#for f in range(0, 20, 1):
#for f in range(0, n_files, 1):


lat_lon_temp = []
lat_temp = []
lon_temp = []
ele_temp = []

gpx_file_temp = ingest_file_list[f]
# read GPX file
with open(gpx_file_temp, 'r') as file:
    gpx = gpxpy.parse(file)
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                #lat_lon_temp.append([point.latitude, point.longitude])
                lon_temp.append([point.longitude])
                lat_temp.append([point.latitude])
                ele_temp.append(point.elevation)

#lat_lon_temp = np.array(lat_lon_temp)  # [deg, deg]
lon_temp = np.array(lon_temp) 
lat_temp = np.array(lat_temp) 
ele_temp = np.array(ele_temp) 

n_points = len(lon_temp)
#print('    read %s points ' %(n_points)) 
n_points_old = n_points

# use Ramer–Douglas–Peucker algorithm to reduce the number of trackpoints
if (use_RDP):
    temp_array = np.hstack([lat_temp, lon_temp, np.arange(0, n_points, 1).reshape(-1, 1)])
    temp_array_new = RDP(temp_array, epsilon) # remove trackpoints less than epsilon meters away from the new track
    index = temp_array_new[:,2].astype(int) # hack
    ele_temp = np.squeeze(ele_temp[index])
    lon_temp = np.squeeze(lon_temp[index])
    lat_temp = np.squeeze(lat_temp[index])
    n_points = len(lon_temp)
    #print('    reduced points from %s to %s ' %(n_points_old, n_points))    
if (f == 0):
    lat_all = lat_temp
    lon_all = lon_temp
    ele_all = ele_temp
else: 
    lat_all = np.hstack([lat_all, lat_temp])
    lon_all = np.hstack([lon_all, lon_temp])
    ele_all = np.hstack([ele_all, ele_temp])
del lon_temp, lat_temp, ele_temp        
n_points_all = len(lat_all)
print('  %s total points ' %(n_points_all))    




  269 total points 


In [6]:

# process gpx w/ rdp
# creates lines from processed
# create lines with dist_min
# plot both 



In [7]:
n_all = np.full([n_points_all], 0, dtype=int)
print('%s total points ' %(n_points_all))    



269 total points 


In [8]:
n_points_thinned = 0
n = 10000
for n in range(0, n_points_all, 1):
    if (n%1000 == 0):
        print('  processing n %s of %s ' %(n, n_points_all)) 
    lat_temp = lat_all[n]
    lon_temp = lon_all[n]
    ele_temp = ele_all[n]
    #print(lat_temp, lon_temp, ele_temp)
    #dist_temp = (lat_all-lat_temp)**2.0 + (lon_all-lon_temp)**2
    #print(np.shape(dist_temp))
    #print(np.shape(dist_temp))
    #print(np.min(dist_temp))
    #print(np.max(dist_temp))
    #dist_temp

    if not (np.isnan(lon_temp)):
        dist_temp = calc_dist_between_one_point_to_all_points(lon_temp, lat_temp, lon_all, lat_all)
        index_close = np.argwhere(dist_temp < dist_min)
        n_nearby_points = len(index_close)
        #print('    found %s nearby points ' %(n_nearby_points)) 
        if (n_nearby_points > 1):
            n_points_thinned = n_points_thinned + n_nearby_points - 1
            lon_avg = np.mean(lon_all[index_close])
            lat_avg = np.mean(lat_all[index_close])
            ele_avg = np.mean(ele_all[index_close])
            #print(lat_temp, lon_temp, ele_temp)
            #print(lat_avg, lon_avg, ele_avg)
            #print(index_close)
            #print(lon_all[index_close])
            #print(lat_all[index_close])
            #print(ele_all[index_close])
            lon_all[index_close] = np.nan
            lat_all[index_close] = np.nan
            ele_all[index_close] = np.nan
            lon_all[n] = lon_avg
            lat_all[n] = lat_avg
            ele_all[n] = ele_avg
            n_all  [n] = n_nearby_points
            del lon_avg, lat_avg, ele_avg
        del dist_temp, index_close, n_nearby_points
        
        
print('%s n_points_thinned ' %(n_points_thinned))    
        

  processing n 0 of 269 
0 n_points_thinned 


In [9]:

print(np.nanmin(n_all))
print(np.nanmax(n_all))


0
0


In [10]:
n_points_all_old = n_points_all
#print(np.shape(lat_all))
mask = ~np.isnan(lat_all)
lon_all = lon_all[mask] 
lat_all = lat_all[mask] 
ele_all = ele_all[mask] 
n_all   =   n_all[mask] 
n_points_all = len(lat_all)
print('reduced points from %s to %s ' %(n_points_all_old, n_points_all))    


reduced points from 269 to 269 


In [11]:
#type(n_all)
#print(np.shape(n_all))
#n_all = np.array(n_all).astype(int)
#print(np.shape(n_all))


In [12]:
print(np.nanmin(n_all))
print(np.nanmax(n_all))
print(np.shape(lon_all))
print(np.shape(n_all))


0
0
(269,)
(269,)


In [25]:
print(dist_min)
dist_min = 10

1.0


In [42]:
# here need to check raw points vs thinned points on a map 

features_tracks_thin = []

n = 1000
n_lines = 0
for n in range(0, n_points_all, 1):
    if (n%1000 == 0):
        print('  processing n %s of %s ' %(n, n_points_all)) 
    lat_temp = lat_all[n]
    lon_temp = lon_all[n]
    ele_temp = ele_all[n]
    n_temp   =   n_all[n]

    dist_temp = calc_dist_between_one_point_to_all_points(lon_temp, lat_temp, lon_all, lat_all)
    # note csmith - here not sure sure what dist_min for lines should be 
    #index_close = np.argwhere(dist_temp < 2.0*dist_min)
    index_close = np.argwhere(dist_temp < 10.0*dist_min)
    n_nearby_points = len(index_close)
    #print('    found %s nearby points' %(n_nearby_points))
    for i in range(0, n_nearby_points, 1):
        i_temp = index_close[i][0]
        #print ('    n %s, i %s, i_temp %s ' %(n, i, i_temp))
        if not (i_temp == n): # skip self 
            n_lines = n_lines + 1
            n_times = max(n_temp, n_all[i_temp]) # must be at least 1
            if (n_times == 0):
                n_times = 1
            line = geojson.LineString([(lon_temp, lat_temp), (lon_all[i_temp], lat_all[i_temp])]) 
            feature = geojson.Feature(geometry=line, properties={'n_times': int('%.0f'%n_times)})
            #feature = geojson.Feature(geometry=line, properties={'n_times': int('%.0f'%n_times)})
            features_tracks_thin.append(feature)
            del n_times, line, feature

feature_collection_tracks_thin = geojson.FeatureCollection(features_tracks_thin)
            
print('%s total lines ' %(n_lines))    
            

  processing n 0 of 269 
1732 total lines 


In [43]:
# create lines from adjacent points 

features_tracks_all = []

n = 1000
n_lines = 0
for n in range(1, n_points_all, 1):
    if (n%1000 == 0):
        print('  processing n %s of %s ' %(n, n_points_all)) 
    lat_temp = lat_all[n]
    lon_temp = lon_all[n]
    ele_temp = ele_all[n]
    n_temp   =   n_all[n]
    n_times = 0
    n_lines = n_lines + 1
    line = geojson.LineString([(lon_all[n], lat_all[n]), (lon_all[n-1], lat_all[n-1])]) 
    feature = geojson.Feature(geometry=line, properties={'n_times': int('%.0f'%n_times)})
    #feature = geojson.Feature(geometry=line, properties={'n_times': int('%.0f'%n_times)})
    features_tracks_all.append(feature)
    del n_times, line, feature
    
feature_collection_tracks_all  = geojson.FeatureCollection(features_tracks_all)

print('%s total lines ' %(n_lines))    


268 total lines 


In [44]:
style_track_all  = lambda x: {'color': '#FC4C02', 'weight': 5} # show some color...
style_track_thin = lambda x: {'color': '#022cfc', 'weight': 5} # show some color...


In [45]:
#fmap = folium.Map(tiles='Stamen Terrain', prefer_canvas=True, disable_3d=True)
fmap = folium.Map(tiles='Stamen Terrain', location=[37.862606, -121.978372], zoom_start=10) # 13 
folium.TileLayer(tiles = 'OpenStreetMap', name='OpenStreetMap', show=False).add_to(fmap)
folium.TileLayer(tiles = 'Stamen Terrain', name='Terrain Map', show=True).add_to(fmap)
cmap = cm.get_cmap('jet') # matplotlib colormap
print('appending features to map ')
#fmap
#folium.GeoJson(feature_collection_tracks_all, style_function=style_track_all, name='track', show=True, smooth_factor=3.0).add_to(fmap)
folium.GeoJson(feature_collection_tracks_thin, style_function=style_track_thin, name='track', show=True, smooth_factor=3.0).add_to(fmap)

#geojson_data_n_times = geojson.FeatureCollection(features_n_times)
#folium.GeoJson(geojson_data_n_times, style_function=style_n_times, tooltip=tooltip_n_times, name='n_times', show=False, smooth_factor=3.0).add_to(fmap)
        
fmap

appending features to map 


In [46]:
#fmap = folium.Map(tiles='Stamen Terrain', prefer_canvas=True, disable_3d=True)
fmap = folium.Map(tiles='Stamen Terrain', location=[37.862606, -121.978372], zoom_start=10) # 13 
folium.TileLayer(tiles = 'OpenStreetMap', name='OpenStreetMap', show=False).add_to(fmap)
folium.TileLayer(tiles = 'Stamen Terrain', name='Terrain Map', show=True).add_to(fmap)
cmap = cm.get_cmap('jet') # matplotlib colormap
print('appending features to map ')
#fmap
folium.GeoJson(feature_collection_tracks_all, style_function=style_track_all, name='track', show=True, smooth_factor=3.0).add_to(fmap)
#folium.GeoJson(feature_collection_tracks_thin, style_function=style_track_thin, name='track', show=True, smooth_factor=3.0).add_to(fmap)

#geojson_data_n_times = geojson.FeatureCollection(features_n_times)
#folium.GeoJson(geojson_data_n_times, style_function=style_n_times, tooltip=tooltip_n_times, name='n_times', show=False, smooth_factor=3.0).add_to(fmap)
        
fmap

appending features to map 


In [ ]:


for feature in geojson_data['features']:
    line = geojson.LineString(feature['geometry']['coordinates'])
    n_times = feature['properties']['n_times']
    features_tracks.append(geojson.Feature(geometry=line))
    features_n_times.append(geojson.Feature(geometry=line, properties={'n_times': n_times}))




In [ ]:
file_name = 'master.geojson'
geojson_write_file = os.path.join(dir_work, file_name)
print('  geojson_write_file is %s ' %(geojson_write_file))
if os.path.isfile(geojson_write_file):
    temp_command = 'rm '+geojson_write_file
    os.system(temp_command)
with open(geojson_write_file, 'w') as file:
    geojson.dump(feature_collection, file)



use_RDP = True
# use Ramer–Douglas–Peucker algorithm to reduce the number of trackpoints
if (use_RDP):
    epsilon = 1 # [m]
    tmp = np.hstack((lat_lon_data, np.arange(0, n_points).reshape((-1, 1)))) # hack
    tmp_new = RDP(tmp, epsilon) # remove trackpoints less than epsilon meters away from the new track
    index = tmp_new[:, 2].astype(int) # hack
    lat_lon_data   = lat_lon_data  [index,:]
    elevation_data = elevation_data[index]
    timestamp_data = timestamp_data[index]
    distance_data  = distance_data [index]
    slope_data     = slope_data    [index]
    speed_data     = speed_data    [index]

n_points = len(slope_data)
print('  read %s points ' %(n_points))    
    
# convert units
if use_SI:
    speed_data = speed_data*3.6 # m/s to km/h
else:
    speed_data = speed_data*2.236936 # m/s to mph

slope_data = abs(slope_data*100) # decimal to %

# create GeoJSON feature collection
features = []
for i in np.arange(1, n_points):
    #print('    processing %s of %s points ' %(i, n_points))    
    #print('    %s, %s, %s, %s ' %(lat_lon_data[i-1, 1], lat_lon_data[i-1, 0], lat_lon_data[i, 1], lat_lon_data[i, 0]))    
    try:
        line = geojson.LineString([(lat_lon_data[i-1, 1], lat_lon_data[i-1, 0]), (lat_lon_data[i, 1], lat_lon_data[i, 0])]) # (lon,lat) to (lon,lat) format
        feature = geojson.Feature(geometry=line, properties = {'elevation': float('%.1f'%elevation_data[i]), 'slope': float('%.1f'%slope_data[i]), 'speed': float('%.1f'%speed_data[i])})
        features.append(feature)
    except:
        print('    ERROR %s of %s points ' %(i, n_points))    

feature_collection = geojson.FeatureCollection(features)
